In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import glob
from tqdm import tqdm
import math
from sklearn.neighbors import KernelDensity
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import folium

In [3]:
def latlonTable2GDF(table: pd.DataFrame, colName_lat: str, colName_lon: str):
    return gpd.GeoDataFrame(table, geometry=gpd.points_from_xy(table[colName_lon], table[colName_lat])).set_crs(6668)
def haversine_distance(coord1, coord2):
    """
    緯度・経度の2点間の直線距離（ハバーサインの公式）
    :param coord1: (lat1, lon1) - 点1の緯度・経度
    :param coord2: (lat2, lon2) - 点2の緯度・経度
    :return: 2点間の距離（単位：メートル）
    """
    R = 6371.0 * 1000  # 地球の半径（メートル）

    lat1, lon1 = map(math.radians, coord1)
    lat2, lon2 = map(math.radians, coord2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

In [4]:
perf_no = "14"
pref_name = "神奈川県"
# 対象地区（横浜市）
target_jcodes = [
    "14101",
    "14102",
    "14103",
    "14104",
    "14105",
    "14106",
    "14107",
    "14108",
    "14109",
    "14110",
    "14111",
    "14112",
    "14113",
    "14114",
    "14115",
    "14116",
    "14117",
    "14118"
]
path = f"Y:\\GIS\\ゼンリンデータ\\建物ポイント\\*\\*"

In [ ]:
target_paths = []
for p in tqdm(glob.glob(path)):
    jcode = p.split("\\")[-1][3:8]
    if jcode in target_jcodes:
        target_paths.append(p)

In [ ]:
dfs = []
for p in tqdm(target_paths):
    dfs.append(pd.read_csv(p,usecols=["市区町村名","大字名","字丁目名","街区","地番・戸番","建物名","階数","建物分類","面積","緯度","経度"],encoding="shift_jis"))
dfs = pd.concat(dfs)

In [ ]:
dfs["address"] = (
        dfs["市区町村名"].fillna('') + "_" +
        dfs["大字名"].fillna('') + "_" +
        dfs["字丁目名"].fillna('') + "_" +
        dfs["街区"].fillna('') + "_" +
        dfs["地番・戸番"].fillna('') + "_" +
        dfs["建物名"].fillna('') + "_" +
        dfs["階数"].fillna('').astype(str)
)
dfs = dfs.filter(items=["建物分類","面積","緯度","経度","address"]).set_axis(["genre","area","latitude","longitude","address"],axis=1)
house = [1001,1002,1003,1004,1005,1006,1007,1008,9999]
non_house = dfs.query("genre not in @house")

In [ ]:
JCODE = "14100"
genre_master = pd.read_csv("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\zenrin_genre_master.csv",usecols=["建物分類","産業大分類","モデル用定義"]).set_axis(["genre","industry_type","behavior_type"],axis=1)
pd.merge(non_house,genre_master,on="genre",how="left").filter(items=["genre","area","latitude","longitude","address","industry_type","behavior_type"]).to_csv(f"C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\zenrin_{JCODE}.csv",index=False)

In [5]:
# non_house.to_csv("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\zenrin_14100.csv")
non_house = pd.read_csv("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\zenrin_14100.csv")

対象範囲にclip

In [6]:
mesh_poly = gpd.read_file("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\mesh\\mesh_geom.shp")

In [7]:
sjoined = latlonTable2GDF(non_house,"latitude","longitude").sjoin(mesh_poly.to_crs(6668),how="left",predicate="intersects").filter(items=["genre","area","latitude","longitude","address","mesh_code","industry_type","behavior_type"])

In [9]:
# 対象地域内
sjoined["abs_attract_score"] = 1 # デフォルトで魅力値を1にセット
sjoined.query("mesh_code == mesh_code").to_csv("C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\poi\\zenrin_building.csv",index=False)

,genre,area,latitude,longitude,address,mesh_code,industry_type,behavior_type
0,3004,159.88,35.501833,139.695914,横浜市鶴見区_朝日町_１丁目__５８_中丸産業運輸（株）_0,NaN,"A_農業，林業, B_漁業 , C_鉱業，採石業，砂利採取業, D_建設業, E_製造業, ...","家庭雑事, 買い物, 行楽・散策, 社会参加"
1,2027,32.14,35.501526,139.696184,横浜市鶴見区_朝日町_１丁目__８_（有）金井板金工業_0,NaN,"A_農業，林業, B_漁業 , C_鉱業，採石業，砂利採取業, D_建設業, E_製造業, ...","家庭雑事, 買い物, 行楽・散策, 社会参加"
2,2010,160.29,35.501559,139.696497,横浜市鶴見区_朝日町_１丁目__１５_セブンイレブン横浜朝日町１丁目店_0,NaN,I_卸売業，小売業,買い物
3,3004,353.38,35.501873,139.697394,横浜市鶴見区_朝日町_１丁目__２０_寛政鉄工（株）_0,NaN,"A_農業，林業, B_漁業 , C_鉱業，採石業，砂利採取業, D_建設業, E_製造業, ...","家庭雑事, 買い物, 行楽・散策, 社会参加"
4,2021,89.84,35.502508,139.697078,横浜市鶴見区_朝日町_１丁目__４２‐１_横浜大東郵便局_0,NaN,"H_運輸業，郵便業, Q_複合サービス事業",家庭雑事
...,...,...,...,...,...,...,...,...
63108,2027,119.39,35.569716,139.568673,横浜市都筑区_あゆみが丘__１０_３３_皆川園_0,NaN,"A_農業，林業, B_漁業 , C_鉱業，採石業，砂利採取業, D_建設業, E_製造業, ...","家庭雑事, 買い物, 行楽・散策, 社会参加"
63109,2018,77.82,35.568800,139.566685,横浜市都筑区_あゆみが丘__１８_２３_あざみ野さくら物語_0,NaN,P_医療，福祉,"家庭雑事, 療養・静養"
63110,2027,70.17,35.568471,139.566492,横浜市都筑区_あゆみが丘__１８_４_しゅしゅあゆみが丘店_0,NaN,"A_農業，林業, B_漁業 , C_鉱業，採石業，砂利採取業, D_建設業, E_製造業, ...","家庭雑事, 買い物, 行楽・散策, 社会参加"
63111,2010,4876.64,35.567359,139.566763,横浜市都筑区_あゆみが丘__１９_７_ニトリ港北ニュータウン店_3,NaN,I_卸売業，小売業,買い物


In [ ]:
sjoined

In [ ]:
# 対象地域外
out_area = sjoined.query("mesh_code != mesh_code").copy()
out_area["eval"] = out_area.apply(
    lambda row: row["area"]/haversine_distance((centLat,centLon),(row["latitude"],row["longitude"])),axis=1)

In [ ]:
cs = out_area.query("genre == 2010").copy()

In [ ]:
# データの読み込み
latlon = np.vstack([cs.latitude, cs.longitude]).T
values = cs["eval"]  # 評価値を格納しているカラム

# カーネル密度推定の実行
kde = KernelDensity(bandwidth=0.01, kernel='gaussian')
kde.fit(latlon, sample_weight=values)  # 評価値をサンプルウェイトとして使用
x_d = np.linspace(latlon[:, 0].min(), latlon[:, 0].max(), 100)
y_d = np.linspace(latlon[:, 1].min(), latlon[:, 1].max(), 100)
xv, yv = np.meshgrid(x_d, y_d)
xy_sample = np.vstack([xv.ravel(), yv.ravel()]).T
density = np.exp(kde.score_samples(xy_sample)).reshape(100, 100)

# 密度分布の可視化
plt.imshow(density, extent=(x_d.min(), x_d.max(), y_d.min(), y_d.max()), origin='lower')
plt.colorbar(label='Density')
plt.show()

# ローカルピークの検出
peaks, _ = find_peaks(density.ravel(), prominence=0.8)  # prominenceは調整可能
peak_positions = np.unravel_index(peaks, density.shape)

# ローカルピークに対応する緯度経度
peak_latlon = np.vstack([x_d[peak_positions[1]], y_d[peak_positions[0]]]).T
print(peak_latlon)

In [ ]:
# foliumの地図作成（最初のピーク地点を中心に設定）
m = folium.Map(location=[peak_latlon[0][0], peak_latlon[0][1]], zoom_start=12)

# ローカルピークを地図に追加
for lat, lon in peak_latlon:
    folium.Marker([lat, lon], popup=f'Peak: ({lat}, {lon})').add_to(m)

# 地図を表示
m

sqlデータの確認

In [10]:
import sqlite3

In [11]:
conn = sqlite3.connect("C:/Users/tora2/IdeaProjects/cityScope/data/database/yokosuka_test.db")
cursor = conn.cursor()
cursor.execute("select * from poi_location")
for r in cursor.fetchall():
    print(r)
conn.close()

('0626e23d-a443-41c9-ae36-23e15490ec27', '2002', 129.48, 35.4324739, 139.6004133, '横浜市南区_井土ケ谷上町__１_１８_品濃屋酒店_0', '5339141833', 'I_卸売業，小売業', '買い物', 1)
('dd8515f2-b43f-451d-8a01-6843f5162f9a', '2021', 2312.88, 35.4328194, 139.60069, '横浜市南区_井土ケ谷上町__１_１_横浜南郵便局_0', '5339141833', 'H_運輸業，郵便業, Q_複合サービス事業', '家庭雑事', 1)
('5dc0b7af-9105-4fa5-884a-5a14bc3ca4f7', '2020', 1290.57, 35.4327994, 139.5994992, '横浜市南区_井土ケ谷上町__２_１_市立井土ヶ谷小学校_4', '5339141744', 'O_教育，学習支援業', '子どもの世話, 授業・学内の活動', 1)
('e98dc51e-a991-4da2-9685-1d147726424e', '2027', 52.17, 35.4326097, 139.5994386, '横浜市南区_井土ケ谷上町__３_２０_アーティス（株）事業本部_3', '5339141744', 'A_農業，林業, B_漁業 , C_鉱業，採石業，砂利採取業, D_建設業, E_製造業, G_情報通信業, K_不動産業，物品賃貸業, N_生活関連サービス業，娯楽業, T_分類不能の産業', '家庭雑事, 買い物, 行楽・散策, 社会参加', 1)
('602ab162-fad3-4139-8abc-137d6251b8c7', '3001', 189.51, 35.4322717, 139.6000069, '横浜市南区_井土ケ谷上町__３_５_のまビル_4', '5339141833', 'A_農業，林業, B_漁業 , C_鉱業，採石業，砂利採取業, D_建設業, E_製造業, G_情報通信業, K_不動産業，物品賃貸業, N_生活関連サービス業，娯楽業, T_分類不能の産業', '家庭雑事, 買い物, 行楽・散策, 社会参加', 1)
('e1000e3b-